In [ ]:
import glob

path = '/Users/kris/data_sources/fdic/BulkReports/'

In [ ]:
for folder in glob.glob(path + '*[!.]'):
    print(f'{folder}')

In [ ]:
folders = glob.glob(path + '*[!.]')
print(f'{folders}')

In [ ]:
import pandas as pd
import datetime

df = pd.DataFrame({'Date': '20240506', 'As_of': datetime.datetime.now()}, index=[0])
df0 = pd.DataFrame(columns=['Date', 'As_of'])
df1 = pd.DataFrame({'Date': '20240506', 'As_of': datetime.datetime.now()}, index=[0])

#df = pd.concat([df, df1], ignore_index=True)

In [ ]:
df0 = pd.concat([df0, df1], ignore_index=True)

In [ ]:
df = pd.concat(df1, df1)

In [ ]:
#type(df['Date'])
#df['Date'].isin(['20240506'])

'20240506' in df['Date'].values

In [ ]:
df.shape[0]

In [ ]:
skipped_dtypes = {'Date': str, 'As_of': datetime.datetime}
skipped_dtypes.keys()

In [ ]:
import time

t = time.time()
period = 60

diff = t - period

current_time = time.strftime('%H:%M:%S', time.localtime(t))
wait_time = time.strftime('%H:%M:%S', time.localtime(diff))

milliseconds_current = int((t - int(t)) * 1000)
milliseconds_diff = int((diff - int(diff)) * 1000)

print(f'current: {current_time}.{milliseconds_current}\nwait: {wait_time}.{milliseconds_diff}\n')


In [ ]:
import time
import datetime

current = time.time()
#print(datetime.datetime.fromtimestamp(current + 3600).strftime("%Y%m%d %H:%M:%S"))
print(f'{time.strftime("%Y%m%d %H:%M:%S",time.gmtime(current + 3600))}')

In [ ]:
#ij = 0
for i in range(10):
    for j in range(5):
        ij +=1
print(ij)

In [ ]:
# Paths to your XBRL file and schema file

bulkReport_path = '/Users/kris/data_sources/fdic/BulkReports/'
schema_base_path = '/Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 09302023_Form051'
#schema_base_path = '/Users/kris/Downloads/_ 09302023_Form051/'

xbrl_file_path = bulkReport_path + 'ABINGTON BANK_61476/ABINGTON BANK_61476_20230930.XBRL'
schema_file_path = schema_base_path + 'call-report051-2023-09-30-v261.xsd'
presentation_file_path = schema_base_path + 'call-report051-2023-09-30-v261-pres.xml'

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = '/Users/kris/data_sources/fdic/BulkReports/ABINGTON BANK_61476/ABINGTON BANK_61476_20230930.XBRL'
#schema_file_path = '/Users/kris/Downloads/_ 09302023_Form051/call-report051-2023-09-30-v261.xsd'

def parse_xsd(schema_path):
    with open(schema_path, 'rb') as schema_file:
        schema_root = etree.XML(schema_file.read())
    return schema_root

def parse_xbrl(xbrl_path):
    with open(xbrl_path, 'rb') as xbrl_file:
        xbrl_root = etree.XML(xbrl_file.read())
    return xbrl_root

def extract_data_from_xbrl(xbrl_root, schema_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def present_data(data):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    print(df)

def main(xbrl_file_path, schema_file_path):
    schema_root = parse_xsd(schema_file_path)
    xbrl_root = parse_xbrl(xbrl_file_path)
    data = extract_data_from_xbrl(xbrl_root, schema_root)
    present_data(data)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def present_data(data):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Optionally, parse and use the presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    
    # Present data
    present_data(data)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_presentation_linkbase(presentation_root):
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    arcrole = 'http://www.xbrl.org/2003/arcrole/parent-child'
    role_refs = presentation_root.xpath('//link:roleRef', namespaces=link_namespace)
    
    presentations = {}
    for role_ref in role_refs:
        role_uri = role_ref.get('roleURI')
        locators = presentation_root.xpath(f'//link:loc[@xlink:role="{role_uri}"]', namespaces=link_namespace)
        arcs = presentation_root.xpath(f'//link:definitionArc[@xlink:arcrole="{arcrole}"]', namespaces=link_namespace)
        
        elements = []
        for arc in arcs:
            from_locator = arc.get('from')
            to_locator = arc.get('to')
            from_element = next((loc for loc in locators if loc.get('label') == from_locator), None)
            to_element = next((loc for loc in locators if loc.get('label') == to_locator), None)
            if from_element is not None and to_element is not None:
                elements.append((from_element.get('{http://www.w3.org/1999/xlink}href'), to_element.get('{http://www.w3.org/1999/xlink}href')))
        presentations[role_uri] = elements
    
    return presentations

def present_data(data, presentations):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    presentation_order = []
    for role_uri, elements in presentations.items():
        for parent, child in elements:
            presentation_order.append(child.split('#')[-1])
    
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    presentations = parse_presentation_linkbase(presentation_root)
    
    # Present data
    present_data(data, presentations)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_presentation_linkbase(presentation_root):
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    xlink_namespace = {'xlink': 'http://www.w3.org/1999/xlink'}
    
    locators = {}
    for loc in presentation_root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    presentation_order = []
    for arc in presentation_root.xpath('//link:presentationArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            presentation_order.append(locators[to_label])
    
    return presentation_order

def present_data(data, presentation_order):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    presentation_order = parse_presentation_linkbase(presentation_root)
    
    # Present data
    present_data(data, presentation_order)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "/path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "/path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "/mnt/data/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    print(f"Parsing XML file: {file_path}")
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    print("Validating XBRL file")
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    print("Extracting data from XBRL")
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_presentation_linkbase(presentation_root):
    print("Parsing presentation linkbase")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    xlink_namespace = {'xlink': 'http://www.w3.org/1999/xlink'}
    
    locators = {}
    for loc in presentation_root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    presentation_order = []
    hierarchy = {}
    for arc in presentation_root.xpath('//link:presentationArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            presentation_order.append(locators[to_label])
            if locators[from_label] not in hierarchy:
                hierarchy[locators[from_label]] = []
            hierarchy[locators[from_label]].append(locators[to_label])
    
    return presentation_order, hierarchy

def present_data(data, presentation_order, hierarchy):
    print("Presenting data")
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    # Display the hierarchical structure
    for parent, children in hierarchy.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    print("Starting main process")
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    presentation_order, hierarchy = parse_presentation_linkbase(presentation_root)
    
    # Present data
    present_data(data, presentation_order, hierarchy)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file, schema file, and taxonomy files
bulkReport_path = '/Users/kris/data_sources/fdic/BulkReports/'
schema_path = '/Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 12312023_Form051/'

xbrl_file_path = os.path.join(bulkReport_path, "ABINGTON BANK_61476/ABINGTON BANK_61476_20231231.XBRL")
schema_files = [
    os.path.join(schema_path, "call-report051-2023-12-31-v265.xsd"),
    os.path.join(schema_path, "concepts.xsd"),
    os.path.join(schema_path, "ffiec-instance-2004-06-10.xsd"),
    os.path.join(schema_path, "ffiec-linkbase-2004-05-14.xsd"),
    os.path.join(schema_path, "mdr_schema.xsd")
]
linkbase_files = {
    "presentation": os.path.join(schema_path, "call-report051-2023-12-31-v265-pres.xml"),
    "calculation": os.path.join(schema_path, "call-report051-2023-12-31-v265-calc.xml"),
    "definition": os.path.join(schema_path, "call-report051-2023-12-31-v265-def.xml"),
    "reference": os.path.join(schema_path, "call-report051-2023-12-31-v265-ref.xml"),
    "label": os.path.join(schema_path, "call-report051-2023-12-31-v265-instr.xml"),
    "ec-mess": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec-mess.xml"),
    "ec": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec.xml")
}

def parse_xml(file_path):
    print(f"Parsing XML file: {file_path}")
    try:
        with open(file_path, 'rb') as file:
            root = etree.XML(file.read())
        return root
    except Exception as e:
        print(f"Failed to parse XML file: {file_path}. Error: {e}")
        raise

def validate_xbrl(xbrl_root, schema_roots):
    print("Validating XBRL file")
    for schema_root in schema_roots:
        print(f"Validating against schema: {schema_root}")
        schema = etree.XMLSchema(schema_root)
        if not schema.validate(xbrl_root):
            print(f"Validation failed against schema: {schema_root}")
            raise ValueError("XBRL file is not valid against the schema.")
        else:
            print(f"Validation succeeded against schema: {schema_root}")
    print("Validation completed")
    return True

def extract_data_from_xbrl(xbrl_root):
    print("Extracting data from XBRL")
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_linkbase(linkbase_file, arc_tag):
    print(f"Parsing {arc_tag} linkbase")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    root = parse_xml(linkbase_file)
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    arcs = []
    for arc in root.xpath(f'//link:{arc_tag}Arc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            arcs.append((locators[from_label], locators[to_label]))
    
    return locators, arcs

def parse_presentation_linkbase(presentation_file):
    locators, presentation_arcs = parse_linkbase(presentation_file, 'presentation')
    presentation_order = []
    hierarchy = {}
    for parent, child in presentation_arcs:
        presentation_order.append(child)
        if parent not in hierarchy:
            hierarchy[parent] = []
        hierarchy[parent].append(child)
    
    return presentation_order, hierarchy

def parse_calculation_linkbase(calculation_file):
    locators, calculation_arcs = parse_linkbase(calculation_file, 'calculation')
    calculations = {}
    for parent, child in calculation_arcs:
        if parent not in calculations:
            calculations[parent] = []
        calculations[parent].append(child)
    
    return calculations

# Function to parse the definition linkbase with additional debug statements
def parse_definition_linkbase(definition_file):
    print(f"Parsing definition linkbase: {definition_file}")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    try:
        root = etree.parse(definition_file)
    except Exception as e:
        print(f"Failed to parse definition linkbase file: {definition_file}. Error: {e}")
        return {}
    
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    print(f"Locators found: {locators}")
    
    definition_arcs = []
    for arc in root.xpath('//link:defArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            definition_arcs.append((locators[from_label], locators[to_label]))
    
    print(f"Definition arcs found: {definition_arcs}")
    
    definitions = {}
    for parent, child in definition_arcs:
        if parent not in definitions:
            definitions[parent] = []
        definitions[parent].append(child)
    
    print(f"Definitions: {definitions}")
    return definitions
    
def parse_additional_linkbases(additional_file):
    # Add logic to parse additional linkbases if needed
    pass

def present_data(data, presentation_order, hierarchy, calculations, definitions):
    print("Presenting data")
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    # Display the hierarchical structure
    print("Hierarchy:")
    for parent, children in hierarchy.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    # Display calculation relationships
    print("\nCalculations:")
    for parent, children in calculations.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    # Display definition relationships
    print("\nDefinitions:")
    for parent, children in definitions.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    print("\nData:")
    print(df)

def main(xbrl_file_path, schema_files, linkbase_files):
    print("Starting main process")
    schema_roots = [parse_xml(file_path) for file_path in schema_files]
    xbrl_root = parse_xml(xbrl_file_path)
    
    #validate_xbrl(xbrl_root, schema_roots)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse linkbases
    presentation_order, hierarchy = parse_presentation_linkbase(linkbase_files['presentation'])
    calculations = parse_calculation_linkbase(linkbase_files['calculation'])
    definitions = parse_definition_linkbase(linkbase_files['definition'])
    
    # Parse additional linkbases if needed
    parse_additional_linkbases(linkbase_files['ec-mess'])
    parse_additional_linkbases(linkbase_files['ec'])
    
    # Present data
    present_data(data, presentation_order, hierarchy, calculations, definitions)

if __name__ == "__main__":
    main(xbrl_file_path, schema_files, linkbase_files)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file, schema file, and taxonomy files
bulkReport_path = '/Users/kris/data_sources/fdic/BulkReports/'
schema_path = '/Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 12312023_Form051/'

xbrl_file_path = os.path.join(bulkReport_path, "ABINGTON BANK_61476/ABINGTON BANK_61476_20230930.XBRL")
schema_files = [
    os.path.join(schema_path, "call-report051-2023-12-31-v265.xsd"),
    os.path.join(schema_path, "concepts.xsd"),
    os.path.join(schema_path, "ffiec-instance-2004-06-10.xsd"),
    os.path.join(schema_path, "ffiec-linkbase-2004-05-14.xsd"),
    os.path.join(schema_path, "mdr_schema.xsd")
]
linkbase_files = {
    "presentation": os.path.join(schema_path, "call-report051-2023-12-31-v265-pres.xml"),
    "calculation": os.path.join(schema_path, "call-report051-2023-12-31-v265-calc.xml"),
    "definition": os.path.join(schema_path, "call-report051-2023-12-31-v265-def.xml"),
    "reference": os.path.join(schema_path, "call-report051-2023-12-31-v265-ref.xml"),
    "label": os.path.join(schema_path, "call-report051-2023-12-31-v265-instr.xml"),
    "ec-mess": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec-mess.xml"),
    "ec": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec.xml")
}

def parse_xml(file_path):
    print(f"Parsing XML file: {file_path}")
    try:
        with open(file_path, 'rb') as file:
            root = etree.XML(file.read())
        return root
    except Exception as e:
        print(f"Failed to parse XML file: {file_path}. Error: {e}")
        raise

def validate_xbrl(xbrl_root, schema_roots):
    print("Validating XBRL file")
    for schema_root in schema_roots:
        print(f"Validating against schema: {schema_root}")
        schema = etree.XMLSchema(schema_root)
        if not schema.validate(xbrl_root):
            print(f"Validation failed against schema: {schema_root}")
            raise ValueError("XBRL file is not valid against the schema.")
        else:
            print(f"Validation succeeded against schema: {schema_root}")
    print("Validation completed")
    return True

def extract_data_from_xbrl(xbrl_root):
    print("Extracting data from XBRL")
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_linkbase(linkbase_file, arc_tag):
    print(f"Parsing {arc_tag} linkbase: {linkbase_file}")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    root = parse_xml(linkbase_file)
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    print(f"Locators found: {locators}")
    
    arcs = []
    for arc in root.xpath(f'//link:{arc_tag}Arc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        #print(f"Processing {arc_tag}Arc from {from_label} to {to_label}")
        if from_label in locators and to_label in locators:
            arcs.append((locators[from_label], locators[to_label]))
    
    print(f"Arcs found in {arc_tag} linkbase: {arcs}")
    return locators, arcs

def parse_presentation_linkbase(presentation_file):
    locators, presentation_arcs = parse_linkbase(presentation_file, 'presentation')
    presentation_order = []
    hierarchy = {}
    for parent, child in presentation_arcs:
        presentation_order.append(child)
        if parent not in hierarchy:
            hierarchy[parent] = []
        hierarchy[parent].append(child)
    
    return presentation_order, hierarchy

def parse_calculation_linkbase(calculation_file):
    print(f"Parsing calculation linkbase: {calculation_file}")
    locators, calculation_arcs = parse_linkbase(calculation_file, 'calculation')
    calculations = {}
    for parent, child in calculation_arcs:
        if parent not in calculations:
            calculations[parent] = []
        calculations[parent].append(child)
    
    print(f"Calculations: {calculations}")
    return calculations

def parse_definition_linkbase(definition_file):
    print(f"Parsing definition linkbase: {definition_file}")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    root = parse_xml(definition_file)
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    print(f"Locators found: {locators}")
    
    definition_arcs = []
    for arc in root.xpath('//link:defArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        print(f"Processing defArc from {from_label} to {to_label}")
        if from_label in locators and to_label in locators:
            definition_arcs.append((locators[from_label], locators[to_label]))
    
    print(f"Definition arcs found: {definition_arcs}")
    
    definitions = {}
    for parent, child in definition_arcs:
        if parent not in definitions:
            definitions[parent] = []
        definitions[parent].append(child)
    
    print(f"Definitions: {definitions}")
    return definitions

def parse_additional_linkbases(additional_file):
    # Add logic to parse additional linkbases if needed
    pass

def present_data(data, presentation_order, hierarchy, calculations=None, definitions=None):
    print("Presenting data")
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    # Display the hierarchical structure
    print("Hierarchy:")
    for parent, children in hierarchy.items():
        print(f"{parent}: {children}")
    
    # Display calculations
    if calculations:
        print("\nCalculations:")
        for parent, children in calculations.items():
            print(f"{parent}: {children}")
    
    # Display definitions
    if definitions:
        print("\nDefinitions:")
        for parent, children in definitions.items():
            print(f"{parent}: {children}")

def main():
    # Parsing the provided definition linkbase
    definitions = parse_definition_linkbase(linkbase_files['definition'])

    # Parsing other necessary linkbases
    calculations = parse_calculation_linkbase(linkbase_files['calculation'])
    presentation_order, hierarchy = parse_presentation_linkbase(linkbase_files['presentation'])

    # Parsing and validating XBRL data
    xbrl_root = parse_xml(xbrl_file_path)
    schema_roots = [parse_xml(schema_file) for schema_file in schema_files]
    #validate_xbrl(xbrl_root, schema_roots)

    # Extracting data from XBRL
    data = extract_data_from_xbrl(xbrl_root)

    # Presenting data
    present_data(data, presentation_order, hierarchy, calculations=calculations, definitions=definitions)

if __name__ == "__main__":
    main()

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import ReadTimeout
from urllib3.util.retry import Retry

class TimeoutHTTPAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        self.timeout = kwargs.pop("timeout", None)
        super().__init__(*args, **kwargs)

    def send(self, request, **kwargs):
        timeout = kwargs.get("timeout")
        if timeout is None:
            kwargs["timeout"] = self.timeout
        
        try:
            response = super().send(request, **kwargs)
        except ReadTimeout as e:
            print(f"ReadTimeout occurred: {e}")
            # Handle the ReadTimeout error here as needed
            raise
        return response

# Usage
def get_session_with_timeout(timeout):
    session = requests.Session()
    
    # Define retries
    retries = Retry(total=3, backoff_factor=0.3, status_forcelist=[500, 502, 503, 504])
    
    # Create an instance of TimeoutHTTPAdapter with a specific timeout
    adapter = TimeoutHTTPAdapter(timeout=timeout, max_retries=retries)
    
    # Mount the adapter to the session for both HTTP and HTTPS
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    return session

# Example usage
if __name__ == "__main__":
    session = get_session_with_timeout(timeout=5)  # 5 seconds timeout
    
    try:
        response = session.get("https://httpbin.org/delay/10")  # This should trigger a timeout
        print(response.status_code)
    except ReadTimeout:
        print("A read timeout occurred.")
    except Exception as e:
        print(f"An error occurred: {Exception.__name__}:: {e}")

In [ ]:
from zeep import Client 
from zeep.wsse.username import UsernameToken


client = Client(wsdl, wsse=UsernameToken(ffiec_un,ffiec_pw))
service = ZeepServiceProxy(client.service, rate_limiter)

response = service.RetrieveFilersSubmissionDateTime(dataSeries = 'Call',
                                                            reportingPeriodEndDate = finlPeriod,
                                                            lastUpdateDateTime = lastUpdate_finlPeriod)

In [ ]:
import FDIC.constants as paths
import os.path
import glob

#path = paths.localPath + paths.folder_BulkReports
path = '/Users/kris/data_sources/fdic/BulkReports/ABBEVILLE BUILDING AND LOAN A STATE CHARTERED SAVINGS BANK_114774'

def get_latest_mod_date(path, file_search_pattern='*'):
    """
    Get the latest modification date of files in a directory.
    """
    latest_date = None
    for file_path in glob.glob(path + file_search_pattern):
        file_date = os.path.getmtime(file_path)
        if latest_date is None or file_date > latest_date:
            latest_date = file_date
    return latest_date


#if get_latest_mod_date(path=path, file_search_pattern='*_master.csv') > get_latest_mod_date(path=path, file_search_pattern='/*.XBRL')

date_master = get_latest_mod_date(path=path, file_search_pattern='/*_master.csv')
date_xbrl = get_latest_mod_date(path=path, file_search_pattern='/*.XBRL')

print(date_master,' || ', date_xbrl)


In [ ]:
import xml.etree.ElementTree as ET
import FDIC.wsio as wsio


filepath = '/Users/kris/data_sources/fdic/Src/Taxonomies/Call_ 12312023_Form051/call-report051-2023-12-31-v265-cap.xml'

def ParseXBRL(self, filepath):
    """
    Parses Call Reports to extract all MDRM item and value from XBRL file
    """
    # Parse XML
    tree  = ET.parse(filepath)
    root = tree.getroot()

    report_values = []
    for child in root:
        #print(child.tag, child.attrib)
        code = child.tag[child.tag.find('}')+1:]
        if len(code) == 8:
            report_values.append([code, child.text])

    parsed_df = pd.DataFrame(report_values, columns = ['MDRM_Item','Value'])
    MDRM_df = wsio.ReadCSV(paths.folder_Orig + paths.filename_MDRM)
    full_df=parsed_df.set_index('MDRM_Item').join(MDRM_df.set_index('MDRM_Item'))
    #print(len(tot[pd.isnull(tot.Item_Name)]))
    full_df.reset_index(level=0, inplace=True)
    return full_df


tree  = ET.parse(filepath)
root = tree.getroot()

report_values = []
for child in root:
    #print(child.tag, child.attrib)
    code = child.tag[child.tag.find('}')+1:]
    if len(code) == 8:
        report_values.append([code, child.text])

parsed_df = pd.DataFrame(report_values, columns = ['MDRM_Item','Value'])
MDRM_df = wsio.ReadCSV(paths.folder_Orig + paths.filename_MDRM)
full_df=parsed_df.set_index('MDRM_Item').join(MDRM_df.set_index('MDRM_Item'))
#print(len(tot[pd.isnull(tot.Item_Name)]))
full_df.reset_index(level=0, inplace=True)






In [ ]:
pres_file_path = '/Users/kris/data_sources/fdic/Src/Taxonomies/Call_ 12312023_Form051/call-report051-2023-12-31-v265-pres.xml'
cap_file_path = '/Users/kris/data_sources/fdic/Src/Taxonomies/Call_ 12312023_Form051/call-report051-2023-12-31-v265-cap.xml'

path = '/Users/kris/data_sources/fdic/Src/Taxonomies/Call_ 12312023_Form051/call-report051-2023-12-31-v265-pres.xml'
pres_search_string = '/*pres.xml'

In [ ]:
def remove_text(input_data):
    texts_to_remove = [
        '{http://www.w3.org/1999/xlink}',
        '{http://www.xbrl.org/2003/linkbase}'
        ]
    
    def removeText(text, texts_to_remove):
        for string in texts_to_remove:
            text = text.replace(string, "")
        return text

    if isinstance(input_data, str):
        return removeText(input_data, texts_to_remove)
    elif isinstance(input_data, dict):
        return {removeText(key, texts_to_remove) if isinstance(key, str) else key: removeText(value, texts_to_remove) if isinstance(value, str) else value
                for key, value in input_data.items()}
    else:
        raise TypeError("Input data must be either a str or a dict")


In [ ]:
import xml.etree.ElementTree as ET
import FDIC.constants as paths
import pandas as pd
import glob
import regex as re


def safe_get_prefix(attribute, key, prefix_length=3):
    value = attribute.get(key)
    return value[:prefix_length] if value else None


def get_ReportAndDate(filename) -> tuple[str, str]:

    #File is formatted as follows:  "call-report051-2023-12-31-v265-pres.xml"
    #search_pattern = '([^/]+$)'
    search_pattern = 'Form[0-9]{3}'
    fn_report = re.search(search_pattern,filename).group()
    #fn_date_split = filename.replace('call-report','').split('-')
    fn_date_split = filename.split('-')
    fn_date = f'{fn_date_split[2]}-{fn_date_split[3]}-{fn_date_split[4]}' if (fn_date_split[2] and fn_date_split[3] and fn_date_split[4]) else None
    return (fn_report, fn_date)


# Print the structure of the presentation file
# These taxonomies have a very particular and nuanced structure
def parse_pres(input_path= paths.localPath + paths.folder_Taxonomies, input_search_pattern= '*/*-pres.xml', input_df= None, output_filepath= paths.folder_Orig + paths.filename_MDRM):

    pres_dtype = {'from':str, 'to':str, 'date':str, 'report':str}
    pres_df = pd.DataFrame(columns=['from', 'to', 'date', 'report']) #, dtype=pres_dtype)

    for file_path in glob.glob(input_path + input_search_pattern):

        file_df = pd.DataFrame(columns=['from', 'to', 'date', 'report']) #, dtype=pres_dtype)

        # Parse the presentation linkbase file
        xml_tree = ET.parse(file_path)
        xml_root = xml_tree.getroot()

        for child in xml_root:
            #print(child.tag, child.attrib)

            if child.tag[child.tag.find('}')+1:] == 'presentationLink':
                for i,ch in enumerate(child):

                    tag = remove_text(ch.tag)
                    attrib = remove_text(ch.attrib)
                    label_prefix = safe_get_prefix(attrib, 'label')
                    from_prefix = safe_get_prefix(attrib, 'from')
                    to_prefix = safe_get_prefix(attrib, 'to')

                    if tag == 'presentationArc' and (label_prefix == 'cc_' or to_prefix == 'cc_'):
                        #att_label = attrib.get("label", "N/A")
                        att_from = attrib.get('from', 'N/A')
                        att_to = attrib.get("to", "N/A")
                        if att_from:
                            att_report, att_date = get_ReportAndDate(file_path) 
                            file_data = pd.DataFrame({'from': att_from, 'to': att_to, 'date':att_date, 'report':att_report}, index=[0]) #, dtype=pres_dtype)
                            file_df = pd.concat([file_df, file_data])
        
        pres_df = pd.concat([pres_df, file_df], ignore_index=True)
    pres_df.to_csv(output_filepath, sep=',', quotechar='"', index= False)  



def parse_cap(input_path= paths.localPath+paths.folder_Taxonomies, input_search_pattern = '*/*-cap.xml', input_df= None, output_filepath= paths.folder_Orig+paths.filename_MDRM):
    cap_dtype = {'from':str, 'to':str, 'label':str, 'text':str}
    cap_df = pd.DataFrame(columns=['from', 'to', 'label', 'text']) #, dtype=cap_dtype)

    for file_path in glob.glob(input_path + input_search_pattern):

        file_df = pd.DataFrame(columns=['from', 'to', 'label', 'text']) #, dtype=cap_dtype)

        # Parse the presentation linkbase file
        xml_tree = ET.parse(file_path)
        xml_root = xml_tree.getroot()

        for child in xml_root:
            #print(child.tag, child.attrib)

            if child.tag[child.tag.find('}')+1:] == 'labelLink':
                for i,ch in enumerate(child):

                    tag = remove_text(ch.tag)
                    attrib = remove_text(ch.attrib)

                    if tag == 'labelArc':
                        att_from = attrib.get('from', 'N/A')
                        att_to = attrib.get('to', 'N/A')
                        
                    if tag == 'label':
                        att_label = attrib.get('label', 'N/A')
                        att_text = ch.text
                        
                        if att_label:
                            file_data = pd.DataFrame({'from': att_from, 'to': att_to, 'label':att_label, 'text':att_text}, index=[0]) #, dtype=pres_dtype)
                            file_df = pd.concat([file_df, file_data])
                            #Reset values of all attribute variables
                            att_from, att_to, att_label, att_text = None, None, None, None


                    #[2 iterations to fill each of these]
                    # check row label = labelArc
                        #set values
                    # check row label = label
                        #set values

                    #[Run every 2 iterations or when all vals are full??]
                    # if labelArc or label
                        #set df values; append to df

        cap_df = pd.concat([cap_df, file_df], ignore_index=True)
    cap_df.to_csv(output_filepath, sep=',', quotechar='"', index= False)  



def merge_dataframes(dataframes_and_mergeColumns: list[tuple[pd.DataFrame, str]], is_filepath=False, how='inner'):
    """
    Merges multiple dataframes into a single dataframe based on specified columns.

    :param dataframes_and_mergeColumns: List of tuples containing either DataFrames or file paths and their corresponding
                                merge column names. 
                                Format: [(df_or_path, merge_column), (df_or_path, merge_column), ...]
    :param filepaths: Boolean indicating if the input is a list of file paths. Default is False.
    :param how: Type of merge to be performed. Default is 'inner'. Other options: 'outer', 'left', 'right'.
    :return: Merged pandas DataFrame.
    """
    if is_filepath:
        # Load DataFrames from CSV files
        dataframes = [(pd.read_csv(df_or_path), col) for df_or_path, col in dataframes_and_mergeColumns]
    else:
        dataframes = dataframes_and_mergeColumns

    # Initialize merged_df with the first DataFrame and its corresponding merge column
    merged_df, left_on = dataframes[0]

    for df, right_on in dataframes[1:]:
        # Merge on specified columns from each DataFrame
        merged_df = pd.merge(merged_df, df, left_on=left_on, right_on=right_on, how=how)
        left_on = right_on  # Update left_on for the next merge


    return merged_df

# Example usage:
# If you have a list of file paths and their corresponding columns:
# merged_df = merge_dataframes([("file1.csv", "col1"), ("file2.csv", "col2"), ("file3.csv", "col3")],
#                              filepaths=True)

# If you have a list of DataFrames and their corresponding columns:
# merged_df = merge_dataframes([(df1, "col1"), (df2, "col2"), (df3, "col3")],
#                              filepaths=False)





In [ ]:
#cap TEST
#path = paths.localPath + paths.folder_Taxonomies
#file_search_pattern = '*/*-cap.xml'
file_search_pattern = ''
path = '/Users/kris/data_sources/fdic/Src/Taxonomies/Call_ 12312023_Form051/call-report051-2023-12-31-v265-cap.xml'
output = 'FDIC/MDRM_cap_test.csv'

parse_cap(input_path=path, input_search_pattern=file_search_pattern, output_filepath=output)


In [ ]:
pres_file = 'FDIC/MDRM_test.csv'
cap_file = 'FDIC/MDRM_cap_test.csv'

pres_df = pd.read_csv(pres_file, delimiter=',')
cap_file = pd.read_csv(cap_file, delimiter=',')

#test = merge_dataframes([pres_df, cap_file], ['from', 'from'])
test = merge_dataframes([(pres_df, 'from'), (cap_file, 'from')])


In [31]:
import pandas as pd
from FDIC import constants as paths

def build_MDRMdict(input_path =paths.localPath+paths.folder_MDRMs+paths.filename_MDRM_src, export_path =paths.folder_Orig + paths.filename_MDRM, filters=None):
    """
    Reads a CSV file, filters the DataFrame based on the given criteria, 
    and exports the result to a new CSV file.

    Parameters:
    - input_path: str, path to the input CSV file.
    - export_path: str, path where the filtered DataFrame should be saved.
    - filters: dict, optional. A dictionary where keys are column names and values 
               are lists of filter criteria.

    Example:
    build_MDRMdict(
        input_path='path/to/MDRM_CSV.csv',
        export_path='path/to/export/MDRM_dict.csv',
        filters={'Reporting Form': ['FFIEC 031', 'FFIEC 041', 'FFIEC 051']})
    """

    def format_str_date(date_string):
        try:
            date_part = date_string.split()[0]
            month, day, year = date_part.split('/')
            return f"{year.zfill(4)}-{month.zfill(2)}-{day.zfill(2)}"
            #df[['Formatted Start Date', 'Formatted End Date']] = df[['Start Date', 'End Date']].apply(lambda x: x.str.split().str[0].str.split('/').apply(lambda parts: f"{parts[2].zfill(4)}-{parts[0].zfill(2)}-{parts[1].zfill(2)}"))
        except:
            return date_string

    # Read the CSV file with headers starting on line 2
    df = pd.read_csv(input_path, header=1, sep=',', quotechar='"') #, parse_dates=['Start Date', 'End Date'])
    df.dropna(axis=1, how='all', inplace=True)

    df[['Start Date', 'End Date']] = df.apply(lambda row: pd.Series([
        format_str_date(row['Start Date']),
        format_str_date(row['End Date'])]),
        axis=1)
    #df[['Start Date', 'End Date']] = df[['Start Date', 'End Date']].apply(lambda x: '-'.join(x.split()[0].split('/')[::-1]).zfill(10))

    # Apply filters if provided
    if filters:
        for column, values in filters.items():
            df = df[df[column].isin(values)]
    
    #Rename columns and format df to needed format and order
    rename_columns={'Confidentiality':'Confidential',
                    'ItemType':'Item Type',
                    'SeriesGlossary':'Series Glossary'}
    df.rename(columns=rename_columns, inplace=True)
    df['MDRM_Item'] = df['Mnemonic'] + df['Item Code'].astype(str)
    df = df[['MDRM_Item', 'Start Date', 'End Date', 'Item Name', 'Confidential', 'Reporting Form']]

    # Export the filtered DataFrame to the specified export path
    df.to_csv(export_path, index=False)


In [32]:
#build_MDRMdict(export_path =paths.folder_Orig + 'MDRM_test.csv' ,filters={'Reporting Form':['FFIEC 031', 'FFIEC 041', 'FFIEC 051']})
build_MDRMdict(export_path =paths.folder_Orig + 'MDRM_test.csv' ,filters={'Reporting Form':['FFIEC 031', 'FFIEC 041', 'FFIEC 051']})

In [ ]:
from datetime import datetime


data = [{'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1979-12-31 00:00:00'),
  'fyearq': 1979,
  'fqtr': 4,
  'revtq': 50.0},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1979-12-31 00:00:00'),
  'fyearq': 1979,
  'fqtr': 4,
  'revtq': 75.0},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1980-03-31 00:00:00'),
  'fyearq': 1980,
  'fqtr': 1,
  'revtq': 53.5},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1980-03-31 00:00:00'),
  'fyearq': 1980,
  'fqtr': 1,
  'revtq': 78.0},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1980-06-30 00:00:00'),
  'fyearq': 1980,
  'fqtr': 2,
  'revtq': 56.71},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1980-06-30 00:00:00'),
  'fyearq': 1980,
  'fqtr': 2,
  'revtq': 85.80000000000001},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1980-09-30 00:00:00'),
  'fyearq': 1980,
  'fqtr': 3,
  'revtq': 60.679700000000004},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1980-09-30 00:00:00'),
  'fyearq': 1980,
  'fqtr': 3,
  'revtq': 94.38000000000002},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1980-12-31 00:00:00'),
  'fyearq': 1980,
  'fqtr': 4,
  'revtq': 66.14087300000001},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1980-12-31 00:00:00'),
  'fyearq': 1980,
  'fqtr': 4,
  'revtq': 95.32380000000002},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1981-03-31 00:00:00'),
  'fyearq': 1981,
  'fqtr': 1,
  'revtq': 70.77073411000002},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1981-03-31 00:00:00'),
  'fyearq': 1981,
  'fqtr': 1,
  'revtq': 104.85618000000002},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1981-06-30 00:00:00'),
  'fyearq': 1981,
  'fqtr': 2,
  'revtq': 72.89385613330002},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1981-06-30 00:00:00'),
  'fyearq': 1981,
  'fqtr': 2,
  'revtq': 110.09898900000003},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1981-09-30 00:00:00'),
  'fyearq': 1981,
  'fqtr': 3,
  'revtq': 75.80961037863203},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1981-09-30 00:00:00'),
  'fyearq': 1981,
  'fqtr': 3,
  'revtq': 121.10888790000004},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1981-12-31 00:00:00'),
  'fyearq': 1981,
  'fqtr': 4,
  'revtq': 77.32580258620467},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1981-12-31 00:00:00'),
  'fyearq': 1981,
  'fqtr': 4,
  'revtq': 133.21977669000006},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1982-03-31 00:00:00'),
  'fyearq': 1982,
  'fqtr': 1,
  'revtq': 77.32580258620467},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1982-03-31 00:00:00'),
  'fyearq': 1982,
  'fqtr': 1,
  'revtq': 133.21977669000006},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1982-06-30 00:00:00'),
  'fyearq': 1982,
  'fqtr': 2,
  'revtq': 77.32580258620467},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1982-06-30 00:00:00'),
  'fyearq': 1982,
  'fqtr': 2,
  'revtq': 135.88417222380005},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1982-09-30 00:00:00'),
  'fyearq': 1982,
  'fqtr': 3,
  'revtq': 85.05838284482515},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1982-09-30 00:00:00'),
  'fyearq': 1982,
  'fqtr': 3,
  'revtq': 144.03722255722806},
 {'tic': 'AAPL',
  'cusip': 'A00000',
  'datadate': Timestamp('1982-12-31 00:00:00'),
  'fyearq': 1982,
  'fqtr': 4,
  'revtq': 97.4033828448251},
 {'tic': 'MSFT',
  'cusip': 'A11111',
  'datadate': Timestamp('1982-12-31 00:00:00'),
  'fyearq': 1982,
  'fqtr': 4,
  'revtq': 178.982222557228}]


In [2]:
import time

print(time.localtime(time.time()))

time.struct_time(tm_year=2024, tm_mon=8, tm_mday=20, tm_hour=16, tm_min=16, tm_sec=26, tm_wday=1, tm_yday=233, tm_isdst=1)
